**Neural Chef Assistance**  
**Name: Darren Jer Shien Yee**  
**Student ID: 31237223**

**Implementation of Baseline 1: RNN without Attention**

In [9]:
## Requirements
!pip3 install nltk
!pip3 install tensorboardX
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from tensorboardX import SummaryWriter
import numpy as np
import gensim.downloader as api
word2vec_model = api.load('word2vec-google-news-300')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cuda


**Implementation of Baseline 1: Language Method imported from RNN code provided in tutorials**

In [10]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS
        self.unique_words = []

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.unique_words.append(word)
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

**Implementation of Baseline 1: Unicode and ASCII method to preprocess data**

In [11]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
# Lowercase only (since numerical is crucial here as opposed to lab code)
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = s.replace('\t', ' ')
    return s

**Implementation of Baseline 1: Method to read data from provided CSV**

In [12]:
import pandas as pd

def readLangs(reverse=False):
    # Read the CSV file
    train_df = pd.read_csv('Cooking_Dataset/Cooking_Dataset/train.csv')
    valid_df = pd.read_csv('Cooking_Dataset/Cooking_Dataset/dev.csv')
    test_df = pd.read_csv('Cooking_Dataset/Cooking_Dataset/test.csv')
    
    train_df.fillna('', inplace=True)
    valid_df.fillna('', inplace=True)
    test_df.fillna('', inplace=True)
    
    train_ingredients = train_df['Ingredients']
    train_recipes = train_df['Recipe']
    valid_ingredients = valid_df['Ingredients']
    valid_recipes = valid_df['Recipe']    
    test_ingredients = test_df['Ingredients']
    test_recipes = test_df['Recipe']
    
    train_pairs = [[normalizeString(ing), normalizeString(rec)] for ing, rec in zip(train_ingredients, train_recipes)]
    valid_pairs = [[normalizeString(ing), normalizeString(rec)] for ing, rec in zip(valid_ingredients, valid_recipes)]
    test_pairs = [[normalizeString(ing), normalizeString(rec)] for ing, rec in zip(test_ingredients, test_recipes)]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang('Recipe')
        output_lang = Lang('Ingredients')
    else:
        input_lang = Lang('Ingredients')
        output_lang = Lang('Recipe')
    return input_lang, output_lang, train_pairs, valid_pairs, test_pairs

**Implementation of Baseline 1: Preprocess data and intialise language using the methods above**

In [13]:
MAX_LENGTH = 150
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]
    
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, train_pairs, valid_pairs, test_pairs = readLangs(reverse)
    print("Read %s sentence pairs" % len(train_pairs))
    train_pairs = filterPairs(train_pairs)
    valid_pairs = filterPairs(valid_pairs)
    test_pairs = filterPairs(test_pairs)
    print("Trimmed to %s sentence pairs" % len(train_pairs))
    print("Counting words...")
    for pair in train_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    for pair in valid_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    for pair in test_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, train_pairs, valid_pairs, test_pairs

input_lang, output_lang, train_pairs, valid_pairs, test_pairs = prepareData('ingredients', 'recipe', False)
print(random.choice(train_pairs))

Read 101340 sentence pairs
Trimmed to 79894 sentence pairs
Counting words...
Counted words:
Ingredients 37056
Recipe 35620
['2 lb smoked halibut 1 c  butter 1    garlic clove 1 pn onion salt 1/4 ts marjoram 1/8 ts thyme 1 pn ground oregano', 'melt butter in saucepan over low heat . add whole garlic , seasonings and herbs . remove from heat and let steep for an hour . place halibut in baking dish . discard garlic and pour herb butter over fish . warm up in microwave or oven at 350 degrees f. for 15 minutes . andrea , cordova , alaska']


**Implementation of Baseline 1: Encoder RNN for seq2seq model without attention**

In [45]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Implementation of Baseline 1: Decoder RNN for seq2seq model without attention**

In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


**Implementation of Baseline 1: Processing methods for train iter**

In [16]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

**Implementation of Baseline 1: Train method for seq2seq model without attention**

In [17]:
teacher_forcing_ratio = 1


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [18]:
def valid(input_tensor, target_tensor, encoder, decoder, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    return loss.item() / target_length


**Implementation of Baseline 1: Time method for train iter method** 

In [19]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

**Implementation of Baseline 1: Train iter method**

In [20]:
def trainIters(encoder, decoder, n_iters,train_pairs,valid_pairs, print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_train_losses = []
    plot_valid_losses = []
    print_train_loss_total = 0  # Reset every print_every
    plot_train_loss_total = 0  # Reset every plot_every
    print_valid_loss_total = 0  # Reset every print_every
    plot_valid_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    valid_pairs = [tensorsFromPair(random.choice(valid_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        training_input_tensor = training_pair[0]
        training_target_tensor = training_pair[1]

        valid_pair = valid_pairs[iter - 1]
        valid_input_tensor = valid_pair[0]
        valid_target_tensor = valid_pair[1]

        train_loss = train(training_input_tensor, training_target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_train_loss_total += train_loss
        plot_train_loss_total += train_loss
        
        valid_loss = valid(valid_input_tensor, valid_target_tensor, encoder,
             decoder, criterion)
        print_valid_loss_total += valid_loss
        plot_valid_loss_total += valid_loss

        if iter % print_every == 0:
            print_train_loss_avg = print_train_loss_total / print_every
            print_train_loss_total = 0
            print_valid_loss_avg = print_valid_loss_total / print_every
            print_valid_loss_total = 0
            print('%s (%d %d%%) Train Loss: %.4f | Validation Loss: %.4f' % (timeSince(start, iter / n_iters),
                                                                             iter, iter / n_iters * 100,
                                                                             print_train_loss_avg, print_valid_loss_avg))
        if iter % plot_every == 0:
            plot_train_loss_avg = plot_train_loss_total / plot_every
            plot_train_losses.append(plot_train_loss_avg)
            plot_train_loss_total = 0
            
            plot_valid_loss_avg = plot_valid_loss_total / plot_every
            plot_valid_losses.append(plot_valid_loss_avg)
            plot_valid_loss_total = 0
    return plot_train_losses,plot_valid_losses


**Implementation of Baseline 1: Show Plot method**

In [21]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
def showPlot(model_name,points1, points2, epochs, plot_every):
    plt.figure()
    fig, ax = plt.subplots()
    # This locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    x_range = np.array(np.arange(0, epochs-1, plot_every))
    plt.plot(x_range,points1, label='Train')  # Plot points1 with label 'Train'
    plt.plot(x_range,points2, label='Valid')  # Plot points2 with label 'Valid'
    plt.legend()  # Add legend
    plt.xlabel('Epochs')  # Label x-axis
    plt.ylabel('Loss')  # Label y-axis
    plt.title(model_name+' Training and Validation Loss')  # Set title for the plot
    plt.show()

**Implementation of Baseline 1: Evaluation methods to check performance of baseline 1**

In [22]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [23]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words= evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

**Implementation of Baseline 1: Creating instance of baseline 1 and training**

In [47]:
hidden_size = 256
n_iter = 10000
plot_every = 100
print_every = 1000
baseline1_encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
baseline1_decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)
baseline1_plot_train_losses,baseline1_plot_valid_losses = trainIters(baseline1_encoder, baseline1_decoder, n_iter ,train_pairs,valid_pairs, print_every = print_every,plot_every=plot_every)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
showPlot('Baseline 1',baseline1_plot_train_losses,baseline1_plot_valid_losses,n_iter,plot_every)

**Implementation of Baseline 2: RNN with Attention**

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        # self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        # self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        #self.out = nn.Linear(self.hidden_size, self.output_size)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        _, hidden = self.gru(embedded, hidden)

        attn_weights = F.softmax(torch.bmm(hidden, encoder_outputs.T.unsqueeze(0)),dim=-1)
        attn_output = torch.bmm(attn_weights, encoder_outputs.unsqueeze(0))

        concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        output = F.log_softmax(self.out(concat_output), dim=1)

        # attn_weights = F.softmax(
        #     self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        # attn_applied = torch.bmm(attn_weights.unsqueeze(0),
        #                          encoder_outputs.unsqueeze(0))

        # output = torch.cat((embedded[0], attn_applied[0]), 1)
        # output = self.attn_combine(output).unsqueeze(0)

        # output = F.relu(output)
        # output, hidden = self.gru(output, hidden)

        # output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


**Implementation of Baseline 2: Train method for RNN with attention**

In [36]:
teacher_forcing_ratio = 1.0


def train_attn(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [46]:
teacher_forcing_ratio = 1.0
def valid_attn(input_tensor, target_tensor, encoder, decoder, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    return loss.item() / target_length


**Implementation of Baseline 2: Train iter method for RNN with attention**

In [49]:
def trainIters_attn(encoder, decoder, n_iters,print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_train_losses = []
    plot_valid_losses = []
    print_train_loss_total = 0  # Reset every print_every
    plot_train_loss_total = 0  # Reset every plot_every
    print_valid_loss_total = 0  # Reset every print_every
    plot_valid_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    validation_pairs = [tensorsFromPair(random.choice(valid_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        training_input_tensor = training_pair[0]
        training_target_tensor = training_pair[1]

        valid_pair = validation_pairs[iter - 1]
        valid_input_tensor = valid_pair[0]
        valid_target_tensor = valid_pair[1]

        train_loss = train_attn(training_input_tensor, training_target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_train_loss_total += train_loss
        plot_train_loss_total += train_loss
        
        valid_loss = valid_attn(valid_input_tensor, valid_target_tensor, encoder,
             decoder, criterion)
        print_valid_loss_total += valid_loss
        plot_valid_loss_total += valid_loss

        if iter % print_every == 0:
            print_train_loss_avg = print_train_loss_total / print_every
            print_train_loss_total = 0
            print_valid_loss_avg = print_valid_loss_total / print_every
            print_valid_loss_total = 0
            print('%s (%d %d%%) Train Loss: %.4f | Validation Loss: %.4f' % (timeSince(start, iter / n_iters),
                                                                             iter, iter / n_iters * 100,
                                                                             print_train_loss_avg, print_valid_loss_avg))
        if iter % plot_every == 0:
            plot_train_loss_avg = plot_train_loss_total / plot_every
            plot_train_losses.append(plot_train_loss_avg)
            plot_train_loss_total = 0
            
            plot_valid_loss_avg = plot_valid_loss_total / plot_every
            plot_valid_losses.append(plot_valid_loss_avg)
            plot_valid_loss_total = 0
    return plot_train_losses,plot_valid_losses


**Implementation of Baseline 2: Initialising instance of RNN with attention**

In [51]:
hidden_size = 256
n_iter = 10000
plot_every = 100
print_every = 100
baseline2_encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters_attn(baseline2_encoder, attn_decoder, 10000, print_every=1000)

4m 59s (- 44m 57s) (1000 10%) Train Loss: 5.0060 | Validation Loss: 5.7207
9m 55s (- 39m 42s) (2000 20%) Train Loss: 4.1286 | Validation Loss: 7.2443
14m 45s (- 34m 26s) (3000 30%) Train Loss: 3.8955 | Validation Loss: 6.5614
19m 38s (- 29m 28s) (4000 40%) Train Loss: 3.7622 | Validation Loss: 6.5764
24m 46s (- 24m 46s) (5000 50%) Train Loss: 3.7299 | Validation Loss: 6.9598
29m 30s (- 19m 40s) (6000 60%) Train Loss: 3.6491 | Validation Loss: 6.4495
34m 13s (- 14m 40s) (7000 70%) Train Loss: 3.6353 | Validation Loss: 6.7779
38m 55s (- 9m 43s) (8000 80%) Train Loss: 3.6191 | Validation Loss: 6.1473
43m 49s (- 4m 52s) (9000 90%) Train Loss: 3.4564 | Validation Loss: 6.6598
48m 41s (- 0m 0s) (10000 100%) Train Loss: 3.5469 | Validation Loss: 6.7105


([6.809228181963516,
  5.640570060866022,
  5.278540322663254,
  4.990064319763482,
  4.8706591112805935,
  4.75482755430752,
  4.564559758050934,
  4.311348940873211,
  4.48589132458738,
  4.354048506982548,
  4.3049282500059824,
  4.370688218039263,
  4.201784072876013,
  3.9883165850411473,
  4.13628815160465,
  4.146663711976898,
  4.08440164926603,
  4.36289762875585,
  3.8208266411115823,
  3.8695859251754148,
  3.9420229549950925,
  3.9496978686822457,
  3.8182430414985884,
  3.992541015913137,
  3.8970326910528614,
  3.885791905455929,
  3.9258169846639235,
  3.9138098370314895,
  3.8073380145087263,
  3.822285724344025,
  3.8025342096082255,
  3.7236479054172054,
  3.7693651632223277,
  3.800639717502825,
  3.7262704433293834,
  3.7947960392653926,
  3.6869427816958673,
  3.646665841541994,
  3.838052037874645,
  3.8333901461361632,
  3.7032628959362968,
  3.664244943960061,
  3.8412705712376134,
  3.6668013015625593,
  3.6943594268916895,
  3.9228384712020348,
  3.71940022883

**Implementation of Baseline 2: Evaluation method for RNN with Attention**

In [52]:
def evaluate_attn(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [55]:
def evaluateRandomly_attn(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(train_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attention= evaluate_attn(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [56]:
evaluateRandomly_attn(baseline2_encoder, attn_decoder)

> 4 fl coconut cream 4 fl chicken stock 2 tb vegetable oil 2    kaffir lime leaves,roughly 1    garlic clove finely chopped -chopped 1 tb red curry paste 4 oz bamboo shoots, cut into 2 tb fish sauce -slivers 1 ts sugar 20    fresh holy basil leaves 6 oz boneless chicken breasts
= in a wok or frying pan heat the oil and fry the garlic until golden brown . add the curry paste and stir well . pour in the warmed coconut cream and stir until it begins to reduce and thicken . add the fish sauce and sugar and stir . add the chicken and cook , stirring constantly , until the meat is opaque . add the stock , stir and cook for one to two min utes or until the chicken is cooked through . stir in the lime leaves , then add the bamboo shoots and basil leaves . stir , cook gently for a final minute and turn into a serving dish .
< in a large skillet , heat the oil and fry the chicken pieces until crisp . add the cream and cook until the onions are soft , about 10 minutes . add the stock , and cook u

**Extension 1: Data Preprocessing**

In [43]:
MAX_LENGTH = 150
import re
def clean_text(text):
    # Define regular expression pattern to match numeric values and units
    amount_pattern = r'\b[\d.]+(?:\s*(?:g|kg|l|ml|tsp|tbsp|cup|pint|quart|oz|c|tb|ts|b|ea|pk|t|can|cups)\b)?'
    # Remove numeric values and units from the text
    text = re.sub(amount_pattern, '', text)

    # Remove irrelevant information like advertisements, website navigation elements, etc.
    # Assuming such information may be enclosed within angle brackets or parentheses
    bracket_pattern = r'<[^>]*>|(\([^)]*\))'
    remove_words = r'-rrb-|-lrb-'
    # Remove text inside angle brackets and parentheses
    cleaned_text = re.sub(bracket_pattern, '', text)
    cleaned_text = re.sub(remove_words, '', cleaned_text)

    # Handle missing data and standardize text format
    cleaned_text = cleaned_text.lower()  # Convert text to lowercase
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)  # Remove punctuation
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespaces
    return cleaned_text

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, train_pairs, valid_pairs, test_pairs = readLangs(reverse)
    print("Read %s sentence pairs" % len(train_pairs))
    train_pairs = filterPairs(train_pairs)
    valid_pairs = filterPairs(valid_pairs)
    test_pairs = filterPairs(test_pairs)
    print("Trimmed to %s sentence pairs" % len(train_pairs))
    print("Counting words...")
    for pair in train_pairs:
        pair[0] = clean_text(pair[0])
        pair[1] = clean_text(pair[1])
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    for pair in valid_pairs:
        pair[0] = clean_text(pair[0])
        pair[1] = clean_text(pair[1])
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    for pair in test_pairs:
        pair[0] = clean_text(pair[0])
        pair[1] = clean_text(pair[1])
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, train_pairs, valid_pairs, test_pairs

input_lang, output_lang, train_pairs, valid_pairs, test_pairs = prepareData('ingredients', 'recipe', False)
print(random.choice(train_pairs))

Read 101340 sentence pairs
Trimmed to 79894 sentence pairs
Counting words...
Counted words:
Ingredients 17999
Recipe 31231
['cream milk egg yolk sugar shot jack daniels dark chocolate melted for chocolate syrup sugar water cocoa powder for soda soda water or until glass is full shot jack daniels', 'to make the ice cream bring milk and cream to a boil in a separate bowl mix the yolks with the sugar then stir in of the milk mixture stir well then add the rest of the milk strain through a fine strainer and add melted chocolate and jack daniel s for the sauce bring water and sugar to a boil then stir in cocoa let cool in a ounce pilsner glass pour in tablespoons syrup then top with scoop ice cream repeat pour in jack daniel s and fill with soda water serve with long dessert spoon and a straw courtesy of john villa of judson grill']


In [46]:
hidden_size = 256
n_iter = 10000
plot_every = 100
print_every = 100
extension1_encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
extension1_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters_attn(extension1_encoder, extension1_decoder, 10000, print_every=1000)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


**Extension 2: Word2Vec**

In [36]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, pretrained_embeddings=None):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        
        # Load pretrained embedding if any
        if pretrained_embeddings is not None:
            self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(pretrained_embeddings))
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [37]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH, pretrained_embeddings=None):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        # Load pretrained embedding if any
        if pretrained_embeddings is not None:
            # Assuming pretrained_embeddings is a numpy array containing Word2Vec embeddings
            self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(pretrained_embeddings))
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(input_size, embedding_size)
        self.attn = nn.Linear(self.hidden_size + self.embedding_size, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size + self.embedding_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [38]:
teacher_forcing_ratio = 1


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [39]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)



In [40]:
embedding_size = word2vec_model.vector_size
# Initialize an empty numpy array for pretrained embeddings
pretrained_embeddings = np.zeros((len(input_lang.unique_words), embedding_size))
# Populate the numpy array with embeddings
for i in range(len(input_lang.unique_words)):
    word = input_lang.unique_words[i]
    if word in word2vec_model:
        pretrained_embeddings[i] = word2vec_model[word]
    else:
        # Handle out-of-vocabulary words (e.g., initialize with zeros)
        pretrained_embeddings[i] = np.zeros(embedding_size)


In [41]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, embedding_size, hidden_size, pretrained_embeddings=pretrained_embeddings).to(device)
decoder = AttnDecoderRNN(embedding_size, hidden_size, output_lang.n_words, dropout_p=0.1, pretrained_embeddings=pretrained_embeddings).to(device)
trainIters(encoder, decoder, 10000, print_every=500)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
